In [24]:
import numpy as np
import pandas as pd
import datetime as dt
from matplotlib import rc
%matplotlib inline
import seaborn as sns
import pandas_profiling as pp
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import RobustScaler
#plt.rc('font', family='NanumGothic', size= 20)
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus']=False
plt.rcParams['font.size']=12
plt.rcParams['font.family']='NanumGothic'
plt.rcParams['xtick.labelsize']=12
plt.rcParams['ytick.labelsize']=12

data1=pd.read_excel('master_data분석용.xlsx')
data=data1

# PREPROCESSING

In [25]:
# 이상치 제거 #:

# 달성률>5000% 제거
# 펀딩기간일==0 제거 - 2개제거
# 마감배송일 -1넘어가면 로그취했을 때 inf 발생

data=data[(data.펀딩기간일!=0)&(data.펀딩기간일<75)]
data=data[data.달성률<5000]
data=data[data['마감배송차이']>-1]

In [26]:
#히스토그램 정규분포에 가까운 독립변수9(펀딩기간일,Strong,Under,문장수,리워즈종류수,마감배송차이,와디즈기사개수,가독성,이미지수)와 종속변수1(달성률) 로그화
beforescale=['펀딩기간일','Strong','Under','문장수','달성률','리워드종류수','마감배송차이','와디즈기사개수','가독성2','이미지수']
afterscale=['펀딩기간일_T','Strong_T','Under_T','문장수_T','달성률_T','리워드종류수_T','마감배송차이_T','와디즈기사개수_T','가독성2_T','이미지수_T']
for i,j in zip(beforescale,afterscale):
    data[j]=preprocessing.scale(np.log(data[i]+1))
    
data['가독성_T']=RobustScaler().fit_transform(data.가독성.values.reshape(-1, 1))

In [33]:
#q1, q2, q3 = np.percentile(data.달성률, [25, 50, 75])
#print(q1,q2,q3)

107.0 281.0 877.0


In [36]:
#분위별 분포 보기위함
#print(len(data[data.달성률<100]))
#print(len(data[(data.달성률<300)&(data.달성률>=100)]))
#print(len(data[(data.달성률<900)&(data.달성률>=300)]))
#print(len(data[(data.달성률<5000)&(data.달성률>=900)]))

1115
1594
1276
1109


# ENCODING - Categorial Variables

In [27]:
# 펀딩시작월 mapping
펀딩시작월_dict = {1:'Jan', 2:'Feb', 3:'Mar',4:'Apr',5:'May',6:'June',7:'July',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'}
data['펀딩시작월'] = data.펀딩시작월.map(펀딩시작월_dict)

In [28]:
categorical_feature_names= ['앵콜펀딩여부','펀딩성공여부']
for col in categorical_feature_names:
    data[col] = data[col].astype("category")

In [29]:
# Multi변수 명목변수화-원핫인코딩
펀딩시작월 = pd.get_dummies(data['펀딩시작월'])
data=pd.concat([data, 펀딩시작월], axis=1)
del data['펀딩시작월']

카테고리 = pd.get_dummies(data['카테고리'])
data=pd.concat([data, 카테고리], axis=1)
del data['카테고리']

펀딩시작요일 = pd.get_dummies(data['펀딩시작요일'])
data=pd.concat([data, 펀딩시작요일], axis=1)
del data['펀딩시작요일']

In [30]:
#좋아요수는 일단 빼보자, 음절수단어수는 제외(다중공선성)

feature_names=['와디즈기사개수_T',
               #'좋아요수',
               '목표금액', 
               '리워드종류수_T',
               '이미지수_T',
               '비디오수',
               '마감배송차이_T',
               '와디즈팔로워수',
               '과거프로젝트수',
               '과거성공프로젝트수',
               '앵콜펀딩여부',
               #'음절수',
               #'단어수',
               '문장수_T',
               'Strong_T',
               'Under_T',
               '가독성_T',
               '가독성2_T', 
               '제목단어수', 
               '펀딩성공여부',
               '디자인소품',
               '반려동물',
               '뷰티',
               '스포츠·모빌리티',
              '여행·레저',
               '테크·가전',
               '패션·잡화',
               '푸드', 
               '홈리빙',
               'Fri',
               'Mon',
               'Sat',
               'Sun',
               'Thu',
               'Tue',
               'Wed',
               '펀딩기간일_T',
               '일펀딩금액',
               '일글수',
              '문장당강조',
               '문장당밑줄',
              'Jan',
              'Feb',
              'Mar',
              'Apr',
              'May',
              'June',
              'July',
              'Aug',
              'Sep',
              'Oct',
              'Nov',
              'Dec']
data_f = data[feature_names]

# REGRESSION

In [31]:
X = data_f
y = data.달성률_T

# 학습용/테스트용 데이터 분리

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

In [32]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 1000)

rf.fit(X_train, y_train)

print('훈련셋점수:',rf.score(X_train, y_train))
print('테스트셋점수:',rf.score(X_test, y_test))

y_pred = rf.predict(X_test)

import sklearn.metrics as mt

#평가 지표 값들을 계산한다.
mse = mt.mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = mt.r2_score(y_test, y_pred)
print("RF선형MSE:{:.3f}\nRMSE: {:.3f}\nRF선형R2: {:.3f}".format(mse,rmse,r2))

#수정결정계수
n = len(X_train)
k = len(X_train.columns)
adj_r2 = 1 - ((1-r2)*(n-1) / (n-k-1))
adj_r2 = 1 - ((1-r2)*(n-1) / (n-k-1))
print('-'*10)
print('RF선형회귀Adjusted R2: {:.4f}'.format(adj_r2))
#print(mt.accuracy_score( y_pred, y_test))
#print(mt.confusion_matrix(y_pred, y_test))

훈련셋점수: 0.9607362466652875
테스트셋점수: 0.7234691964499287
RF선형MSE:0.296
RMSE: 0.544
RF선형R2: 0.723
----------
RF선형회귀Adjusted R2: 0.7195


In [38]:
# RF Regressor 하이퍼 파라미터 튜닝 - GridSearchCV
from sklearn.model_selection import GridSearchCV

params_rf = {
    'n_estimators': [100, 300,1000],
    'max_depth': [3, 6], 
    }

grid_rf = GridSearchCV(rf, params_rf, 
                      cv = 5, n_jobs = -1, 
                      )

grid_rf.fit(X_train, y_train)
grid_y_pred = grid_rf.predict(X_test)
print(grid_rf.best_estimator_)

mse = mt.mean_squared_error(y_test, grid_y_pred)
rmse = np.sqrt(mse)
r2 = mt.r2_score(y_test, grid_y_pred)
print("grid_RF선형MSE:{:.3f}\ngrid_RMSE: {:.3f}\ngrid_RF선형R2: {:.3f}".format(mse,rmse,r2))

#pd.DataFrame({'속성명':X.columns, '중요도':grid_rf.feature_importances_}).sort_values('중요도',ascending=False)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=6,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
grid_RF선형MSE:0.304
grid_RMSE: 0.552
grid_RF선형R2: 0.716


AttributeError: 'GridSearchCV' object has no attribute 'feature_importances_'

In [14]:
#다중선형회귀 실시
import sklearn.linear_model as lm
import sklearn.metrics as mt
import numpy as np

reg=lm.LinearRegression().fit(X_train,y_train)
y_pred = reg.predict(X_test)

#오차 및 결정계수
mse = mt.mean_squared_error(y_test, y_pred) # 테스트예측값
rmse = np.sqrt(mse)
r2 = mt.r2_score(y_test, y_pred)
print('선형회귀MSE: {:3f}\nRMSE: {:.3f}\n선형회귀R2: {:.3f}'.format(mse, rmse, r2))

#수정결정계수
n = len(X_train)
k = len(X_train.columns)
adj_r2 = 1 - ((1-r2)*(n-1) / (n-k-1))
adj_r2 = 1 - ((1-r2)*(n-1) / (n-k-1))
print('-'*10)
print('선형회귀 Adjusted R2: {:.4f}'.format(adj_r2))

#coefs = pd.Series(data=np.round(reg.coef_,3),index=features_.columns)
#coefs_sort = coefs.sort_values(ascending=False)
#print(coefs_sort)
#print('절편:', round(reg.intercept_, 3))
#np.set_printoptions(suppress=False)

선형회귀MSE: 0.325095
RMSE: 0.570
선형회귀R2: 0.696
----------
선형회귀 Adjusted R2: 0.6921


In [40]:
#릿지 회귀실시  - alpha= [0, 0.01, 1, 10, 100]
import sklearn.linear_model as lm
alphas = [0,0.01,1,10,100]
for a in alphas:
    ridge=lm.Ridge(alpha = a)
    reg=ridge.fit(X_train,y_train)
    y_pred = reg.predict(X_test)

    #오차 및 결정계수
    mse = mt.mean_squared_error(y_test, y_pred) # 테스트예측값
    rmse = np.sqrt(mse)
    r2 = mt.r2_score(y_test, y_pred)
    print('Alpha={}일때: '.format(a))
    print('릿지MSE: {:3f}\nRMSE: {:.3f}\n릿지R2: {:.3f}'.format(mse, rmse, r2))

    #수정결정계수
    n = len(X_train)
    k = len(X_train.columns)
    adj_r2 = 1 - ((1-r2)*(n-1) / (n-k-1))
    adj_r2 = 1 - ((1-r2)*(n-1) / (n-k-1))
    print('-'*10)
    print('릿지Adjusted R2: {:.4f}'.format(adj_r2))
    print('-'*20)

Alpha=0일때: 
릿지MSE: 0.325632
RMSE: 0.571
릿지R2: 0.696
----------
릿지Adjusted R2: 0.6916
--------------------
Alpha=0.01일때: 
릿지MSE: 0.325632
RMSE: 0.571
릿지R2: 0.696
----------
릿지Adjusted R2: 0.6916
--------------------
Alpha=1일때: 
릿지MSE: 0.325690
RMSE: 0.571
릿지R2: 0.696
----------
릿지Adjusted R2: 0.6915
--------------------
Alpha=10일때: 
릿지MSE: 0.326525
RMSE: 0.571
릿지R2: 0.695
----------
릿지Adjusted R2: 0.6907
--------------------
Alpha=100일때: 
릿지MSE: 0.344576
RMSE: 0.587
릿지R2: 0.678
----------
릿지Adjusted R2: 0.6736
--------------------


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.00427e-19): result may not be accurate.
  overwrite_a=True).T
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=9.98667e-18): result may not be accurate.
  overwrite_a=True).T
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=9.6651e-17): result may not be accurate.
  overwrite_a=True).T


In [41]:
#라쏘 회귀 실시 - alpha= [0, 0.01, 1, 10, 100]

import sklearn.linear_model as lm
alphas = [0,0.01,1,10,100]
for a in alphas:
    Lasso=lm.Lasso(alpha = a)
    reg=Lasso.fit(X_train,y_train)
    y_pred = reg.predict(X_test)

    #오차 및 결정계수
    mse = mt.mean_squared_error(y_test, y_pred) # 테스트예측값
    rmse = np.sqrt(mse)
    r2 = mt.r2_score(y_test, y_pred)
    print('Alpha={}일때: '.format(a))
    print('라쏘MSE: {:3f}\nRMSE: {:.3f}\n라쏘R2: {:.3f}'.format(mse, rmse, r2))

    #수정결정계수
    n = len(X_train)
    k = len(X_train.columns)
    adj_r2 = 1 - ((1-r2)*(n-1) / (n-k-1))
    adj_r2 = 1 - ((1-r2)*(n-1) / (n-k-1))
    print('-'*10)
    print('라쏘Adjusted R2: {:.4f}'.format(adj_r2))
    print('-'*20)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:478: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Alpha=0일때: 
라쏘MSE: 0.325632
RMSE: 0.571
라쏘R2: 0.696
----------
라쏘Adjusted R2: 0.6916
--------------------
Alpha=0.01일때: 
라쏘MSE: 0.332644
RMSE: 0.577
라쏘R2: 0.689
----------
라쏘Adjusted R2: 0.6849
--------------------
Alpha=1일때: 
라쏘MSE: 0.951004
RMSE: 0.975
라쏘R2: 0.112
----------
라쏘Adjusted R2: 0.0992
--------------------
Alpha=10일때: 
라쏘MSE: 1.013356
RMSE: 1.007
라쏘R2: 0.054
----------
라쏘Adjusted R2: 0.0402
--------------------
Alpha=100일때: 
라쏘MSE: 1.013399
RMSE: 1.007
라쏘R2: 0.054
----------
라쏘Adjusted R2: 0.0401
--------------------


In [42]:
#엘라스틱넷 회귀 실시
elastic = lm.ElasticNet(alpha = 0.01, l1_ratio=0.5)
el_reg=ridge.fit(X_train,y_train)
y_pred = el_reg.predict(X_test)

r2=mt.r2_score(y_test,y_pred)

print('엘라스틱넷 R2: {:.4f}'.format(r2))
print('엘라스틱넷 Adjusted R2: {:.4f}'.format(adj_r2))

#오차 및 결정계수
mse = mt.mean_squared_error(y_test, y_pred) # 테스트예측값
rmse = np.sqrt(mse)
r2 = mt.r2_score(y_test, y_pred)
print('엘라스틱넷MSE: {:3f}\nRMSE: {:.3f}\n엘라스틱R2: {:.3f}'.format(mse, rmse, r2))
#수정결정계수
n = len(X_train)
k = len(X_train.columns)
adj_r2 = 1 - ((1-r2)*(n-1) / (n-k-1))
adj_r2 = 1 - ((1-r2)*(n-1) / (n-k-1))
print('-'*10)
print('엘라스틱넷Adjusted R2: {:.4f}'.format(adj_r2))

엘라스틱넷 R2: 0.6782
엘라스틱넷 Adjusted R2: 0.0401
엘라스틱넷MSE: 0.344576
RMSE: 0.587
엘라스틱R2: 0.678
----------
엘라스틱넷Adjusted R2: 0.6736


In [49]:
# 엘라스틱넷회귀: 하이퍼 파라미터 튜닝 - GridSearchCV
from sklearn.model_selection import GridSearchCV

params_el = {
    'alpha' : [0,0.3,0.5,0.7,1]
    }

grid_el = GridSearchCV(el_reg, params_el, 
                      cv = 5, n_jobs = -1, 
                      )

grid_el.fit(X_train, y_train)
grid_y_pred = grid_el.predict(X_test)
print(grid_el.best_estimator_)

mse = mt.mean_squared_error(y_test, grid_y_pred)
rmse = np.sqrt(mse)
r2 = mt.r2_score(y_test, grid_y_pred)
print("grid_EL회귀 MSE:{:.3f}\ngrid_EL회귀 RMSE: {:.3f}\ngrid_EL회귀 R2: {:.3f}".format(mse,rmse,r2))

#pd.DataFrame({'속성명':X.columns, '중요도':grid_rf.feature_importances_}).sort_values('중요도',ascending=False)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)
grid_EL회귀 MSE:0.326
grid_EL회귀 RMSE: 0.571
grid_EL회귀 R2: 0.696


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=9.98667e-18): result may not be accurate.
  overwrite_a=True).T


# 변수추가

In [23]:
# 새로운 컬럼(12개) 로그화
# 시작날짜/마감날짜의 일과 그 달 1,5,10,15,20,25일 차이
# 월급일과의 차이가 적을수록 달성률분포가 높게 이루질수도. (가정) 
# 월급일이 다 다를 수 있어서 6개(1,5,10,15,20,25)씩 만들어봄.

addfeatures=['시작1일차이','시작5일차이','시작10일차이','시작15일차이','시작20일차이','시작25일차이','마감1일차이','마감5일차이','마감10일차이',\
             '마감15일차이','마감20일차이','마감25일차이']
addfeatures_scaled=['시작1일차이_T','시작5일차이_T','시작10일차이_T','시작15일차이_T','시작20일차이_T','시작25일차이_T','마감1일차이_T','마감5일차이_T','마감10일차이_T',\
                   '마감15일차이_T','마감20일차이_T','마감25일차이_T']

for i,j in zip(addfeatures,addfeatures_scaled):
    data[j]=preprocessing.scale(np.log(data[i]+1))

In [8]:
feature_names=['와디즈기사개수_T',
               #'좋아요수',
               '목표금액', 
               '리워드종류수_T',
               '이미지수_T',
               '비디오수',
               '마감배송차이_T',
               '와디즈팔로워수',
               '과거프로젝트수',
               '과거성공프로젝트수',
               '앵콜펀딩여부',
               #'음절수',
               #'단어수',
               '문장수_T',
               'Strong_T',
               'Under_T',
               '가독성_T',
               '가독성2_T', 
               '제목단어수', 
               '펀딩성공여부',
               '디자인소품',
               '반려동물',
               '뷰티',
               '스포츠·모빌리티',
              '여행·레저',
               '테크·가전',
               '패션·잡화',
               '푸드', 
               '홈리빙',
               'Fri',
               'Mon',
               'Sat',
               'Sun',
               'Thu',
               'Tue',
               'Wed',
               '펀딩기간일_T',
               '일펀딩금액',
               '일글수',
              '문장당강조',
               '문장당밑줄',
              'Jan',
              'Feb',
              'Mar',
              'Apr',
              'May',
              'June',
              'July',
              'Aug',
              'Sep',
              'Oct',
              'Nov',
              'Dec',
              '시작1일차이_T',
              '시작5일차이_T',
              '시작10일차이_T',
              '시작15일차이_T',
              '시작20일차이_T',
              '시작25일차이_T',
              '마감1일차이_T',
              '마감5일차이_T',
              '마감10일차이_T',
              '마감15일차이_T',
              '마감20일차이_T',
              '마감25일차이_T']
data_f = data[feature_names]

In [9]:
X = data_f
y = data.달성률_T

# 학습용/테스트용 데이터 분리

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

In [10]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 1000)

rf.fit(X_train, y_train)

print('훈련셋점수:',rf.score(X_train, y_train))
print('테스트셋점수:',rf.score(X_test, y_test))

y_pred = rf.predict(X_test)

import sklearn.metrics as mt

#평가 지표 값들을 계산한다.
mse = mt.mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = mt.r2_score(y_test, y_pred)

print("MSE:{:.3f}\nRMSE: {:.3f}\nR2: {:.3f}".format(mse,rmse,r2))
#print(mt.accuracy_score( y_pred, y_test))
#print(mt.confusion_matrix(y_pred, y_test))

pd.DataFrame({'속성명':X.columns, '중요도':rf.feature_importances_}).sort_values('중요도', ascending=False)

훈련셋점수: 0.9600213053698861
테스트셋점수: 0.721007283986373
MSE:0.299
RMSE: 0.547
R2: 0.721


,속성명,중요도
16,펀딩성공여부,0.579809
35,일글수,0.069162
6,와디즈팔로워수,0.046817
34,일펀딩금액,0.036156
3,이미지수_T,0.021155
10,문장수_T,0.015507
5,마감배송차이_T,0.013642
0,와디즈기사개수_T,0.012839
36,문장당강조,0.012829
33,펀딩기간일_T,0.012787
